In [4]:
import os
import json
import pandas as pd
import traceback

In [5]:
from langchain.chat_models import ChatOpenAI

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
KEY = os.getenv("OPENAI_API_KEY")

In [8]:
# llm = ChatOpenAI(
#     openai_api_key=KEY,
#     model_name="openai/gpt-3.5-turbo",
#     temperature=0.7
# )

In [9]:
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=KEY,
    temperature=0.7
)

/tmp/ipykernel_11490/141744126.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


In [46]:
# llm

In [11]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [12]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    
    "2": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    
    "3": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    
}

In [13]:
template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [14]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=template
    )


In [15]:
# quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [16]:
from langchain.schema.runnable import RunnablePassthrough

# Quiz Chain
quiz_chain = quiz_generation_prompt | llm

In [17]:
template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of teh question and give a complete analysis of the quiz if the students
will be able to unserstand the questions and answer them. Only use at max 50 words for complexity analysis. 
if the quiz is not at par with the cognitive and analytical abilities of the students,\
update tech quiz questions which needs to be changed  and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}


Check from an expert English Writer of the above quiz:
"""

In [18]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=template2
)

In [19]:
# Quiz Review Chain
review_chain = quiz_evaluation_prompt | llm

In [20]:
# review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [21]:
from langchain.schema.runnable import RunnableLambda, RunnableMap


# Define Final RunnableMap
generate_evaluate_chain = RunnableMap({
    "quiz": quiz_chain,  # Runs quiz_chain and stores output as "quiz"
    "subject": lambda x: x["subject"]  # Pass "subject" directly
}) | review_chain


In [22]:
# ✅ Fix: Ensure both outputs are returned
def run_review_chain(input_data):
    """Runs review_chain and ensures both quiz and review are returned"""
    quiz_output = input_data["quiz"]
    review_output = review_chain.invoke({"quiz": quiz_output, "subject": input_data["subject"]})
    return {"quiz": quiz_output, "review": review_output}

In [23]:
# ✅ Fix: Ensure both quiz and review are captured
generate_evaluate_chain = RunnableMap({
    "quiz": quiz_chain,  # Generate Quiz
    "subject": lambda x: x["subject"],  # Pass subject
}) | RunnableLambda(run_review_chain)  # Run Review and Return Both


In [24]:
file_path = r"/home/harshit433/mcqgen/data.txt"

In [25]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [26]:
TEXT

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\n\nFrom a theoretical viewpoint, probably ap

In [27]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [28]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [30]:
# Execute with Callback
with get_openai_callback() as cb:
    response = generate_evaluate_chain.invoke({
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    })

In [31]:
print(f"Total Tokens: {cb.total_tokens}") 
print(f"Prompt Tokens: {cb.prompt_tokens}") 
print(f"Completion Tokens: {cb.completion_tokens}" )
print(f"Total Cost: {cb.total_cost}") 

Total Tokens: 2117
Prompt Tokens: 1739
Completion Tokens: 378
Total Cost: 0.0033645000000000003


In [32]:
response

{'quiz': AIMessage(content='{\n"1": {\n"mcq": "What is machine learning?",\n"options": {\n"a": "The study of artificial intelligence concerned with developing and studying statistical algorithms",\n"b": "The study of computer hardware",\n"c": "The study of human cognitive processes",\n"d": "The study of quantum computing"\n},\n"correct": "a"\n},\n"2": {\n"mcq": "Who coined the term \'machine learning\'?",\n"options": {\n"a": "Donald Hebb",\n"b": "Tom M. Mitchell",\n"c": "Arthur Samuel",\n"d": "Walter Pitts"\n},\n"correct": "c"\n},\n"3": {\n"mcq": "What is the purpose of machine learning algorithms for stock trading?",\n"options": {\n"a": "To predict future outcomes based on models developed",\n"b": "To classify data based on models developed",\n"c": "To analyze human cognitive processes",\n"d": "To create computer vision models"\n},\n"correct": "a"\n},\n"4": {\n"mcq": "What is the foundation of machine learning?",\n"options": {\n"a": "Data mining",\n"b": "Computer vision",\n"c": "Stati

In [33]:
quiz = response.get("quiz")

In [34]:
import json
from langchain_core.messages.ai import AIMessage

In [35]:
quiz

AIMessage(content='{\n"1": {\n"mcq": "What is machine learning?",\n"options": {\n"a": "The study of artificial intelligence concerned with developing and studying statistical algorithms",\n"b": "The study of computer hardware",\n"c": "The study of human cognitive processes",\n"d": "The study of quantum computing"\n},\n"correct": "a"\n},\n"2": {\n"mcq": "Who coined the term \'machine learning\'?",\n"options": {\n"a": "Donald Hebb",\n"b": "Tom M. Mitchell",\n"c": "Arthur Samuel",\n"d": "Walter Pitts"\n},\n"correct": "c"\n},\n"3": {\n"mcq": "What is the purpose of machine learning algorithms for stock trading?",\n"options": {\n"a": "To predict future outcomes based on models developed",\n"b": "To classify data based on models developed",\n"c": "To analyze human cognitive processes",\n"d": "To create computer vision models"\n},\n"correct": "a"\n},\n"4": {\n"mcq": "What is the foundation of machine learning?",\n"options": {\n"a": "Data mining",\n"b": "Computer vision",\n"c": "Statistics and

In [36]:
# Convert AIMessage content to JSON
quiz_dict = json.loads(quiz.content)  # Parse string into dictionary

# Print formatted JSON output
print(json.dumps(quiz_dict, indent=4))

{
    "1": {
        "mcq": "What is machine learning?",
        "options": {
            "a": "The study of artificial intelligence concerned with developing and studying statistical algorithms",
            "b": "The study of computer hardware",
            "c": "The study of human cognitive processes",
            "d": "The study of quantum computing"
        },
        "correct": "a"
    },
    "2": {
        "mcq": "Who coined the term 'machine learning'?",
        "options": {
            "a": "Donald Hebb",
            "b": "Tom M. Mitchell",
            "c": "Arthur Samuel",
            "d": "Walter Pitts"
        },
        "correct": "c"
    },
    "3": {
        "mcq": "What is the purpose of machine learning algorithms for stock trading?",
        "options": {
            "a": "To predict future outcomes based on models developed",
            "b": "To classify data based on models developed",
            "c": "To analyze human cognitive processes",
            "d": "To cre

In [37]:
type(quiz)

langchain_core.messages.ai.AIMessage

In [38]:
# ✅ Print the complete response
print("Generated Quiz:\n", response["quiz"])
print("\nEvaluation Review:\n", response["review"])


Generated Quiz:
 content='{\n"1": {\n"mcq": "What is machine learning?",\n"options": {\n"a": "The study of artificial intelligence concerned with developing and studying statistical algorithms",\n"b": "The study of computer hardware",\n"c": "The study of human cognitive processes",\n"d": "The study of quantum computing"\n},\n"correct": "a"\n},\n"2": {\n"mcq": "Who coined the term \'machine learning\'?",\n"options": {\n"a": "Donald Hebb",\n"b": "Tom M. Mitchell",\n"c": "Arthur Samuel",\n"d": "Walter Pitts"\n},\n"correct": "c"\n},\n"3": {\n"mcq": "What is the purpose of machine learning algorithms for stock trading?",\n"options": {\n"a": "To predict future outcomes based on models developed",\n"b": "To classify data based on models developed",\n"c": "To analyze human cognitive processes",\n"d": "To create computer vision models"\n},\n"correct": "a"\n},\n"4": {\n"mcq": "What is the foundation of machine learning?",\n"options": {\n"a": "Data mining",\n"b": "Computer vision",\n"c": "Statist

In [39]:
# This is an Overall Chain where we run the two chains in Sequence
# generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
#                                         output_variables=["quiz", "review"], verbose=True,)

In [40]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [41]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: The study of artificial intelligence concerned with developing and studying statistical algorithms | b: The study of computer hardware | c: The study of human cognitive processes | d: The study of quantum computing',
  'Correct': 'a'},
 {'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Donald Hebb | b: Tom M. Mitchell | c: Arthur Samuel | d: Walter Pitts',
  'Correct': 'c'},
 {'MCQ': 'What is the purpose of machine learning algorithms for stock trading?',
  'Choices': 'a: To predict future outcomes based on models developed | b: To classify data based on models developed | c: To analyze human cognitive processes | d: To create computer vision models',
  'Correct': 'a'},
 {'MCQ': 'What is the foundation of machine learning?',
  'Choices': 'a: Data mining | b: Computer vision | c: Statistics and mathematical optimization | d: Natural language processing',
  'Correct': 'c'},
 {'MCQ': 'In what field does machine learn

In [42]:
df=pd.DataFrame(quiz_table_data)

In [43]:
df

,MCQ,Choices,Correct
0,What is machine learning?,a: The study of artificial intelligence concer...,a
1,Who coined the term 'machine learning'?,a: Donald Hebb | b: Tom M. Mitchell | c: Arthu...,c
2,What is the purpose of machine learning algori...,a: To predict future outcomes based on models ...,a
3,What is the foundation of machine learning?,a: Data mining | b: Computer vision | c: Stati...,c
4,In what field does machine learning find appli...,a: Agriculture | b: Medicine | c: Natural lang...,d


In [45]:
df.to_csv("machinelearning.csv",index=False)